# PNK data wrangling

Open, standardize and manipulate Excel files containing research data from the PROMET study (PNK Database 2).

## Read all the relevant Excel input files into Pandas data frames

### Import raw files

Read the original files provided by Pronokal. They are converted to Pandas data frames for preprocessing and analysis. 

Building Dask dataframes could also be considered, as working with those should be faster. 

In [8]:
import pandas as pd
import dask.dataframe as dd

# Define input paths for raw input data
# path_to_patients = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_10Mar25_intact/pac.xlsx"
# path_to_medical_records = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_10Mar25_intact/exp.xlsx"
# path_to_prescriptions = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_10Mar25_intact/pre.xlsx"
path_to_measurements = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_10Mar25_intact/mes.xlsx"
# path_to_alleles = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_10Mar25_intact/ale.xlsx"

# Read the excels to pandas data frames
# patients_df = pd.read_excel(path_to_patients)
# medical_records_df = pd.read_excel(path_to_medical_records)
# prescriptions_df = pd.read_excel(path_to_prescriptions)
measurements_df = pd.read_excel(path_to_measurements)
# alleles_df = pd.read_excel(path_to_alleles)

# Convert pandas data frames to dask data frames
# patients_ddf = dd.from_pandas(patients_df, npartitions=1)
# medical_records_ddf = dd.from_pandas(medical_records_df, npartitions=1)
# prescriptions_ddf = dd.from_pandas(prescriptions_df, npartitions=1)
# measurements_ddf = dd.from_pandas(measurements_df, npartitions=1)
# alleles_ddf = dd.from_pandas(alleles_df, npartitions=1)

## Standardize the variable names and layout of each data frame

Before merging the different data frames, standardize variable names in each data frame, as well as the database structure to the extent possible. 

Standardize the variable names using English expressions and including units wherever possible. 

Make sure each data frame contains every relevant identifier if possible, as in the raw files, different IDs are present. The 'Prescriptions/pre' file contains 3 out of the 4 identifiers used. 

Omit any clinically irrelevant or too incomplete columns from each data frame. Save the standardized files containing all variables separately from the standardized, cleaned files that contain the relevant columns only. 

Later, the standardized and cleaned data frames should be integrated into an SQL database. 

### Create output directories

Folders where the tables will be saved after standardization and preprocessing. 

In [5]:
import os

# Define output directories for standardized files
# path_to_patients_standard = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/patients_standard/"
# path_to_medical_records_standard = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/medical_records_standard/"
# path_to_prescriptions_standard = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/prescriptions_standard/"
path_to_measurements_standard = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/measurements_standard/"
# path_to_alleles_standard = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/alleles_standard/"

# Create the directories if they do not exist
# os.makedirs(path_to_patients_standard, exist_ok=True)
# os.makedirs(path_to_medical_records_standard, exist_ok=True)
# os.makedirs(path_to_prescriptions_standard, exist_ok=True)
# os.makedirs(path_to_measurements_standard, exist_ok=True)
# os.makedirs(path_to_alleles_standard, exist_ok=True)

### Standardize pre -> prescriptions

Use consistently formatted variable names in English. 


Omit any variables that are clinically irrelevant. 

Characteristics of the data frame:
- 14 170 patients, 113 709 prescriptions
- long format, several rows belonging to the same patient (not sure why - for this, it needs to be defined, what constitutes 1 prescription)
- clinically relevant variables: 'patient_id', 'sex', 'genomics_available', 'genomics_sample_id', 'patient_date', 'birth_date', 'country'
- missing IDs: genomics_sample_id

In [7]:
def standardize_prescriptions(prescriptions_df):
    # Rename columns and reorder them
    prescriptions_df = prescriptions_df.rename(columns={
        "Nombre": "prescription_id",
        "Fecha de creación": "prescription_registration_date",
        "Expediente": "medical_record_id",
        "Pack 1, 2, 3 Pnk": "pack_123_pnk",
        "Vit D 1000 UI": "vit_d_1000iu",
        "Pelo, piel y uñas (60) Comprimido": "hair_skin_nails",
        "VIT C 1000mg.(100) Cáps. Vegetales": "vit_c_1000mg",
        "Pack 1, 2, 3 PnKExpert K-Line": "pack_123_pnkexpert_kline",
        "Cromo + Biotina + Ginseng americano": "chromium_biotine_ginseng",
        "Id Paciente": "patient_id",
        "Genomics": "genomics",
        "Genomics: Fecha Prescripción médico": "genomics_prescription_date",
        "Genomics: Fecha Compra paciente": "genomics_purchase_date",
        "Genomics: Fecha Relizacion prueba": "genomics_sampling_date",
        "Genomics: Fecha Recultados": "genomics_results_date",
        "Pack 1, 2, 3 PnKDiaproKal": "pack_123_diaprokal",
        "Retiradas < Totales": "purchased_meals_<_prescribed",
        "HMB K-Line": "hmb_kline",
        "Omega Balance": "omega_balance",
        "VIT E 400UI (268mg.) Aceite (50) Perlas": "vit_e_400iu",
        "Cellulite Gel": "cellulite_gel",
        "Colágeno VitalProtein": "collagene_vitalprotein",
        "Biotina 300mcg. (100) Comprimidos": "biotine_300mcg",
        "Vit D 4000 UI": "vit_d_4000iu",
        "Stick HMB Plus": "hmbplus_stick",
        "Pack 4, 5 PnKExpert": "pack_45_pnkexpert",
        "Drenacel-K": "drenacel_k",
        "Magnesio": "magnesium",
        "Sodio": "sodium",
        "Pack 4, 5 PnKDiaproKal": "pack_45_diaprokal",
        "Pack 4, 5 Pnk": "pack_45_pnk",
        "DHA Vita": "dha_vita",
        "Natur Transit": "natur_transit",
        "Stick Ca + Mg + Vitaminas y minerales": "calcium_magnesium_vitamines_minerals_stick",
        "VIT D3 1000UI (25mcg.) (100) Cáps. Blandas": "vit_d3_1000iu_soft_capsules",
        "VIT D3 4000 UI (100mcg.) (60) VCaps.": "vit_d3_4000iu_capsules",
        "HbA1C (%)": "hba1c_prescription",
        "Female Multiple (Complejo para la mujer) (60) Comp": "female_multivitamin",
        "Potasio": "potassium",
        "Pack 4, 5 PnKExpert K-Line": "pack_45_pnkexpert_kline",
        "G.A.B.A. 500mg (50) Cáps.Veg.": "gaba_500mg",
        "Aero Relax Natur": "aero_relax_natur",
        "Vitaminas": "vitamins",
        "Glucemia T0 o en ayunas (mg/dl)": "fasting_glucose_t0",
        "VIT B12 1000mcg. (Cianocobalamina) (100) Comp. Mas": "vit_b12_1000mcg",
        "Citrato de magnesio (60) Comprimidos": "magnesium_citrate",
        "Pack 1, 2, 3 PnKExpert": "pack_123_pnkexpert",
        "Simbiotics 2: B animalis lactis (Pasos 4, 5)": "symbiotics2_b_animalis",
        "Calcio": "calcium",
        "Unicomplex": "unicomplex",
        "AVA. 40 + ACIDOF. (60) Caps. Vegetal": "probiotics_ava40",
        "Simbiotics 1: B Longum (Pasos 1, 2, 3)": "symbiotics1_b_longum",
        "Método": "method",
        "Paso": "step",
        "Fecha fin validez": "prescription_validity_end_date",
        "Tomas consumidas": "consumed_meals",
        "Tomas pendientes": "pending_meals",
        "Tomas por día": "daily_meals",
        "Tomas totales": "total_meals",
        "Días de validez": "prescription_validity_days",
        "Fecha y hora prescripción": "prescription_creation_date",
        "Gasto Acumulado": "cumulative_expenses",
        "Médico": "prescribing_doctor"
    })
    # Reorder columns
    column_order = [
        "patient_id", "prescription_id", "medical_record_id", "prescribing_doctor",
        "prescription_creation_date", "prescription_registration_date", "prescription_validity_end_date",
        "prescription_validity_days", "method", "step", "consumed_meals", "pending_meals",
        "daily_meals", "total_meals", "purchased_meals_<_prescribed", "cumulative_expenses",
        "genomics", "genomics_prescription_date", "genomics_purchase_date", "genomics_sampling_date",
        "genomics_results_date", "hba1c_prescription", "fasting_glucose_t0", "pack_123_pnk",
        "pack_123_pnkexpert", "pack_123_pnkexpert_kline", "pack_123_diaprokal", "pack_45_pnk",
        "pack_45_pnkexpert", "pack_45_pnkexpert_kline", "pack_45_diaprokal", "vitamins",
        "vit_b12_1000mcg", "biotine_300mcg", "vit_d_1000iu", "vit_d_4000iu", "vit_d3_1000iu_soft_capsules",
        "vit_d3_4000iu_capsules", "vit_c_1000mg", "vit_e_400iu", "female_multivitamin",
        "calcium_magnesium_vitamines_minerals_stick", "chromium_biotine_ginseng", "sodium", "calcium",
        "potassium", "magnesium", "magnesium_citrate", "hair_skin_nails", "hmb_kline", "hmbplus_stick",
        "cellulite_gel", "collagene_vitalprotein", "drenacel_k", "natur_transit", "gaba_500mg",
        "dha_vita", "omega_balance", "aero_relax_natur", "unicomplex", "symbiotics1_b_longum",
        "symbiotics2_b_animalis", "probiotics_ava40"
    ]
    prescriptions_df = prescriptions_df[column_order]
    prescriptions_df = prescriptions_df.sort_values(by='patient_id')
    return prescriptions_df

# Create and save a standardized data frame with all variables included
prescriptions_standard = standardize_prescriptions(prescriptions_df.copy())
prescriptions_standard.to_excel(path_to_prescriptions_standard + 'prescriptions_standard.xlsx', index=False)
prescriptions_standard

# Drop specified columns from prescription_df
columns_to_drop = [
    "prescribing_doctor", "genomics_purchase_date", "genomics_sampling_date", 
    "genomics_results_date", "hba1c_prescription", "fasting_glucose_t0", 
    "vitamins", "vit_b12_1000mcg", "biotine_300mcg", "vit_d3_1000iu_soft_capsules", 
    "vit_d3_4000iu_capsules", "vit_c_1000mg", "vit_e_400iu", "female_multivitamin", 
    "calcium_magnesium_vitamines_minerals_stick", "chromium_biotine_ginseng", 
    "calcium", "potassium", "magnesium_citrate", "hair_skin_nails", "hmb_kline", 
    "hmbplus_stick", "collagene_vitalprotein", "drenacel_k", "natur_transit", 
    "gaba_500mg", "aero_relax_natur", "probiotics_ava40"
]
prescriptions_standard_clean = prescriptions_standard.drop(columns=columns_to_drop)

# Create and save a standardized data frame with only the relevant variables included
prescriptions_standard_clean.to_excel(path_to_prescriptions_standard + 'prescriptions_standard_clean.xlsx', index=False)
prescriptions_standard_clean.head()


,patient_id,prescription_id,medical_record_id,prescription_creation_date,prescription_registration_date,prescription_validity_end_date,prescription_validity_days,method,step,consumed_meals,...,vit_d_1000iu,vit_d_4000iu,sodium,magnesium,cellulite_gel,dha_vita,omega_balance,unicomplex,symbiotics1_b_longum,symbiotics2_b_animalis
43959,9896A74CEB969,146C624D15241,12902F42A682D,2021-12-22 08:56:09,2021-12-22 09:20:16,2022-01-06 08:56:09,15,PNK,Paso 1 - PNK - Activa,77.0,...,No,No,No,No,No,No,No,No,No,No
41063,9896A74CEB969,1450B74D15241,12902F42A682D,2021-11-23 11:47:00,2021-11-23 13:06:17,2021-12-08 11:47:00,15,PNK,Paso 1 - PNK - Activa,71.0,...,No,No,No,No,No,No,No,No,No,No
9924,9896A74CEB969,12A5604D15241,10ABF442A682D,2021-01-31 12:31:00,2021-02-01 10:27:17,2021-02-15 12:31:00,15,PNK,Paso 1 - PNK - Activa,77.0,...,No,No,No,No,No,No,No,No,No,No
98646,9896A74CEB969,1709924D15241,1385C142A682D,2023-06-08 13:23:50,2023-06-08 11:40:19,2023-06-23 13:23:50,15,PNK,Paso 1 - PNK - Activa,75.0,...,No,No,No,No,No,No,No,No,No,No
94073,9896A74CEB969,16C8CA4D15241,1385C142A682D,2023-04-27 20:26:50,2023-04-28 08:30:19,2023-05-12 20:26:50,15,PNK,Paso 1 - PNK - Activa,77.0,...,No,No,No,No,No,No,No,No,No,No


### Standardize pac -> patients

Use consistently formatted variable names in English. 

Order columns in a logical sense, order rows by patient_id in ascending order. 

Omit any variables that are clinically irrelevant. 

Characteristics of the data frame:
- 14 710 patients
- 81%:19% female:male ratio
- 2579 genomics sample IDs, some patients have genomics sample IDs with the indication that genomics is NOT available (unsure why)
- missing IDs: medical_record_id, prescription_id



In [15]:
def standardize_patients(patients_df):
    # Rename columns according to the provided dictionary
    patients_df = patients_df.rename(columns={
        'Id Paciente': 'patient_id',
        'Fecha de creación': 'patient_record_creation_date',
        'Compra Online': 'online_purchase',
        'Sexo': 'sex',
        'Patient Collective': 'patient_collective',
        'NAV Company Lower': 'country',
        'Fecha de nacimiento': 'birth_date',
        'GDPR-4': 'gdpr4',
        'Fecha GDPR-4': 'gdpr4_date',
        'GDPR-10': 'gdpr10',
        'Fecha GDPR-10': 'gdpr10_date',
        'genomics_available': 'genomics_available',
        'Muestra': 'genomics_sample_id'
    })
    
    # Reorder columns and sort rows
    patients_df = patients_df[['patient_id', 'sex', 'genomics_available', 'genomics_sample_id', 
                               'patient_record_creation_date', 'birth_date', 'country', 'patient_collective', 
                               'online_purchase', 'gdpr4', 'gdpr4_date', 'gdpr10', 'gdpr10_date']]
    patients_df = patients_df.sort_values(by='patient_id', ascending=True)
    return patients_df

# Create and save a standardized data frame with all variables included
patients_standard = standardize_patients(patients_df.copy())
patients_standard.to_excel(path_to_patients_standard + 'patients_standard.xlsx', index=False)
patients_standard

# Drop clinically irrelevant columns
columns_to_drop = ['patient_collective', 'online_purchase', 'gdpr4', 'gdpr4_date', 'gdpr10', 'gdpr10_date']
patients_standard_clean = patients_standard.drop(columns=columns_to_drop)

# Create and save the standardized data frame with only the relevant variables included
patients_standard_clean.to_excel(path_to_patients_standard + 'patients_standard_clean.xlsx', index=False)
patients_standard_clean

# In a subsequent step, try to add missing identifiers to the data frame.
# Keep in mind to retain the original structure of the data frame if possible, 
# without letting the way of importing IDs from the prescriptions data frame alter the size of this one. 
    # Add missing identifiers from the prescriptions data frame
    # ids_except_genomics = prescriptions_standard_clean.drop(columns = ['prescription_creation_date', 'prescription_registration_date',
    #     'prescription_validity_end_date', 'prescription_validity_days',
    #     'method', 'step', 'consumed_meals', 'pending_meals', 'daily_meals',
    #     'total_meals', 'purchased_meals_<_prescribed', 'cumulative_expenses',
    #     'genomics', 'genomics_prescription_date', 'pack_123_pnk',
    #     'pack_123_pnkexpert', 'pack_123_pnkexpert_kline', 'pack_123_diaprokal',
    #     'pack_45_pnk', 'pack_45_pnkexpert', 'pack_45_pnkexpert_kline',
    #     'pack_45_diaprokal', 'vit_d_1000iu', 'vit_d_4000iu', 'sodium',
    #     'magnesium', 'cellulite_gel', 'dha_vita', 'omega_balance', 'unicomplex',
    #     'symbiotics1_b_longum', 'symbiotics2_b_animalis'])
    # patients_df = patients_df.merge(ids_except_genomics, on='patient_id', how='inner')


,patient_id,sex,genomics_available,genomics_sample_id,patient_record_creation_date,birth_date,country
0,9896A74CEB969,V,No,NaN,2019-02-13 14:31:29,1991-02-04 00:00:00,es
1,9896AA4CEB969,V,No,NaN,2019-02-13 14:31:31,1964-02-20 00:00:00,es
2,9896E04CEB969,V,No,NaN,2019-02-13 14:32:01,1975-01-19 00:00:00,es
3,9896E54CEB969,V,No,NaN,2019-02-13 14:32:03,1978-05-10 00:00:00,es
4,9896EA4CEB969,V,No,NaN,2019-02-13 14:32:05,1974-01-17 00:00:00,es
...,...,...,...,...,...,...,...
14165,9AA7F84CEB969,M,No,NaN,2024-02-23 17:00:26,1980-09-04 00:00:00,it
14166,9AA8264CEB969,M,No,NaN,2024-02-26 18:10:20,1984-05-12 00:00:00,it
14167,9AA8294CEB969,V,No,NaN,2024-02-26 18:30:17,1975-10-16 00:00:00,es
14168,9AA8314CEB969,M,No,NaN,2024-02-27 08:40:27,1992-07-15 00:00:00,es


### Standardize exp -> medical_records

Use consistently formatted variable names in English. 

Add patient IDs. 

Order columns in a logical sense, order rows by patient_id in ascending order. 

Omit any variables that are clinically irrelevant.

Add patient IDs and calculate the duration of each intervention. 

One medical record ID corresponds to one weight loss intervention, and a single patient in many cases undergoes several interventions. Separate the patients that underwent a single intervention from those who did multiple. 

Characteristics of the data frame:
- 20 942 individual medical records of 14 170 patients - 9713 patients have 1, 4457 patients have multiple medical records, ie. interventions
- waist circumference data available from about 65% of patients, but the time of registering that variable needs to be confirmed, as it is not obvious whether this is a baseline or a final value. Also, there are about 7700 outlier measurements with values under 10. 
- eating behavior and eating-related emotional values are available from around 8-10% of patients, but the fidelity of these self-reported data points is low. The time of registering them also needs to be confirmed. 
- missing IDs: prescription_id, genomics_sample_id - unsure about how to adequately merge these data frames, as prescriptions is very long, with 113 709 prescription IDs, and alleles has about 2500 genomics sample IDs, so I still need to up my skills a bit in that regard. 

In [27]:
"""
STANDARDIZE: reorder columns and convert variable names to English
"""

def standardize_medical_records(medical_records_df):
    # Rename columns in medical_records_df
    medical_records_df = medical_records_df.rename(columns={
        'Expediente': 'medical_record_id',
        'Fecha de creación': 'medical_record_creation_date',
        'Método': 'pnk_method',
        'Médico recomienda actividad física': 'physical_activity_recommended',
        'Num comidas al día': 'nr_meals_day',
        'Comedor emocional': 'emotional_eating',
        'Objetivo': 'goal',
        'Sensación saciedad': 'satiety',
        'Actividad física': 'physical_activity',
        'Tipo diabetes': 'diabetes_type',
        'Tabaquismo': 'smoking',
        'Sensación hambre': 'hunger',
        'Hábito picar': 'snacking',
        'Causa no actividad física': 'physical_inactivity_cause',
        'Frecuencia actividad física': 'physical_activity_frequency',
        'Sensación apetito': 'appetite',
        'Recursos cocina': 'cooking_resources',
        'Lugar comida': 'eating_location',
        'Tipos cocción': 'cooking_techniques',
        'Medicacion': 'medications',
        'Edad': 'age',
        'Edad al momento de la creación del expediente': 'age_when_creating_record',
        'Control cantidades': 'quantity_control',
        'Fecha de cierre': 'medical_record_closing_date',
        'Control impulso': 'impulse_control',
        'Fecha de nacimiento': 'birth_date',
        'Estatura (mts)': 'height_m',
        'Perímetro cintura obj (cm)': 'wc_goal_cm',
        'Perímetro cintura (cm)': 'wc_cm_confirm_time',
        'Comedor Emocional Valor': 'emotional_eating_value',
        'Pedidos Expediente': 'orders_in_medical_record',
        'Visitas Expediente': 'dietitian_visits',
        'causa del aumento de peso': 'weight_gain_cause'
    })
    # Reorder columns in the data frame       
    medical_records_df = medical_records_df[[
        'medical_record_id',
        'medical_record_creation_date',
        'medical_record_closing_date',
        'birth_date',
        'age',
        'age_when_creating_record',
        'height_m',
        'wc_cm_confirm_time',
        'pnk_method',
        'nr_meals_day',
        'orders_in_medical_record',
        'dietitian_visits',
        'goal',
        'wc_goal_cm',
        'physical_activity_recommended',
        'physical_activity',
        'physical_activity_frequency',
        'physical_inactivity_cause',
        'weight_gain_cause',
        'diabetes_type',
        'smoking',
        'medications',
        'emotional_eating',
        'satiety',
        'hunger',
        'snacking',
        'emotional_eating_value',
        'quantity_control',
        'impulse_control',
        'cooking_resources',
        'eating_location',
        'cooking_techniques'
    ]]
    # Sort rows by patient_id
    medical_records_df = medical_records_df.sort_values(by='medical_record_id', ascending=True)
    return medical_records_df

# Create and save a standardized data frame with all variables included
medical_records_standard = standardize_medical_records(medical_records_df.copy())
medical_records_standard.to_excel(os.path.join(path_to_medical_records_standard, 'medical_records_standard.xlsx'), index=False)
medical_records_standard

"""
CLEAN: Remove irrelevant variables, add relevant but missing variables (Patient ID, intervention duration)
"""

# Remove specified columns from medical_record_df
columns_to_drop = [
    'nr_meals_day', 'goal', 'wc_goal_cm', 'physical_activity_recommended', 
    'diabetes_type', 'snacking', 'cooking_resources', 'eating_location', 'cooking_techniques'
]
medical_records_standard_clean = medical_records_standard.drop(columns=columns_to_drop)

# Add patient IDs to the medical records data frame, and calculate the length of each intervention recorded in each medical record
# First, create the data frame with the IDs and the medical record dates separately
def map_records_to_patients(prescriptions_standard_clean, medical_records_standard_clean):
    # Step 1: Retrieve patient and medical record IDs from prescriptions
    patient_and_medical_ids = prescriptions_standard_clean[['patient_id', 'medical_record_id']].drop_duplicates(subset=['medical_record_id'])
    # Step 2: Retrieve medical record creation dates from medical records
    medical_record_dates = medical_records_standard_clean[['medical_record_id', 'medical_record_creation_date', 'medical_record_closing_date']]
    # Step 3: Merge the two DataFrames to include patient_id, medical_record_id, and medical_record_creation_date
    patients_and_records = patient_and_medical_ids.merge(medical_record_dates, on='medical_record_id', how='left')
    # Step 4: Calculate the length of each intervention
    # Convert dates to datetime format if they are not already
    patients_and_records['medical_record_creation_date'] = pd.to_datetime(patients_and_records['medical_record_creation_date'])
    patients_and_records['medical_record_closing_date'] = pd.to_datetime(patients_and_records['medical_record_closing_date'])
    # Calculate the duration in days
    patients_and_records['intervention_duration_days'] = (
        patients_and_records['medical_record_closing_date'] - patients_and_records['medical_record_creation_date']
    ).dt.days
    return patients_and_records
patient_ids_with_medical_record_ids = map_records_to_patients(prescriptions_standard_clean, medical_records_standard_clean)

# Then, merge the separately created ID database with medical_records_standard_clean

# Perform the merge operation
medical_records_standard_clean = patient_ids_with_medical_record_ids.merge(
    medical_records_standard_clean,
    on=['medical_record_id', 'medical_record_creation_date', 'medical_record_closing_date'],
    how='inner'
)

# Create and save the standardized data frame with all variables included
medical_records_standard_clean.to_excel(os.path.join(path_to_medical_records_standard, 'medical_records_standard_clean.xlsx'), index=False)
medical_records_standard_clean

"""
SPLIT: Separate patients with one medical record from those with multiple records; 
multiple records mean diet restart or several subsequent interventions. 
"""

# Logic: sort by patient_id and date, group by patient_id, and filter groups with one vs more elements
sorted_records = medical_records_standard_clean.sort_values(
    by=["patient_id", "medical_record_creation_date"]
)

# Separate and save single-record patients
single_record_patients = sorted_records.groupby("patient_id").filter(
    lambda group: len(group) == 1
)
single_record_patients.to_excel(os.path.join(path_to_medical_records_standard, 'single_record_patients.xlsx'), index=False)

# Separate and save multiple-record patients
multiple_records_patients = sorted_records.groupby("patient_id").filter(
    lambda group: len(group) > 1
)
multiple_records_patients.to_excel(os.path.join(path_to_medical_records_standard, 'multiple_records_patients.xlsx'), index=False)



### Standardize mes -> measurements IMPORTANT: CONTINUE WITH FINDING OUT THE RESTART/RECORD ID STUFF

Use consistently formatted variable names in English. 

Omit any variables that are clinically irrelevant. 

Exclude about 5000 outliers who have an initial weight of under 50 kg, or an initial BMI of under 25. 

Characteristics of the data frame: 
- long format, with different measurement time points contained in rows not columns, so that one patient has multiple rows with their data
- 296 129 rows with all the measurements
- data from 14 710 total patients, before excluding outliers
- high number of outliers in the following columns: weight, BMI, BMR, fat%, but outliers are present in all cases, most likely due to serious data collection issues
- missing IDs: prescription ID, medical record ID, genomics sample ID

OUTLIERS ISSUE: 

- I filtered the measurements data frame trying to get rid of outliers of weight values of 25 kg and so on
- the filtering logic was: group by patient IDs and sort date, so we get all the measurments in order of occurrence from each patient; after, remove any groups entirely where the first value is an outlier (under 50 kg or 25 BMI).
- In this logic, a group was only removed if the first measurement is an outlier, as I did not want to remove people who achieved a healthy weight over the course of the intervention, just wanted to remove people whose first measures were already too low
- however, a lot of 25 kg measurements remained. In these cases, normal descending trends were followed by a block of 25 kg measurements, and the date differed. For example, 77->65 kg over the course of 4 months, and then after 7 months, a new block from the same patient with 25kg values
- this made me think of trying to pull the medical record IDs from the patients, adding them to the measurements database, and using those values to sort too, being able to get rid of certain dietary attempts of these individuals
- I changed the medical records database cleaning logic to map patient IDs to the medical record IDs, and it works well
- However, there's no direct link between a concrete medical record ID and patient ID that could be used to map medical record IDs to the measurements data frame, only if a less exact, date-based logic is applied, and that looks pretty complicated on a first (very superficial) sight
- alternatively, the restarter patients can be excluded entirely. I think that would be best for now, hopefully the genetics study subjects and the guys with behav data all finished the program for the first time and had no return. 

In [ ]:

"""
STANDARDIZE: reorder columns and convert variable names to English
"""
def standardize_measurements(measurements_df):
    # Rename columns to standardized English names
    measurements_df = measurements_df.rename(columns={
        'Id Paciente': 'patient_id',
        'weight': 'weight_kg',
        'imc': 'bmi',
        'water': 'water_%',
        'muscle': 'muscle_%',
        'bmr': 'bmr_kcal',
        'visceral': 'vat_%',
        'bone': 'bone_%',
        'ica': 'ica_unknown',
        'icc': 'icc_unknown',
        'fat': 'fat_%',
        'type': 'type_unknown',
        'bia': 'bia_unknown',
        'created_at': 'measurement_date'
    })
    # Reorder columns
    measurements_df = measurements_df[[
        'patient_id',
        'measurement_date',
        'weight_kg',
        'bmi',
        'bmr_kcal',
        'fat_%',
        'vat_%',
        'muscle_%',
        'water_%',
        'bone_%',
        'ica_unknown',
        'icc_unknown',
        'type_unknown',
        'bia_unknown'
    ]]
    # Sort rows by patient_id and measurement_date
    measurements_df = measurements_df.sort_values(by=['patient_id', 'measurement_date'])
    return measurements_df

# Create and save a standardized data frame with all variables included
measurements_standard = standardize_measurements(measurements_df.copy())
measurements_standard.to_excel(os.path.join(path_to_measurements_standard, 'measurements_standard.xlsx'), index=False)
measurements_standard

"""
CLEAN: omit clinically irrelevant variables, remove outliers - ALTHOUGH THAT NEEDS CONFIRMATION, as the data is not yet fully understood; 
some outliers due to erroneous measurements we hope be able to recover.
"""
# Omit clinically irrelevant columns, create and save the standardized data frame with only the relevant variables
measurements_standard_clean = measurements_standard.drop(columns=['bone_%', 'ica_unknown', 'icc_unknown', 'type_unknown', 'bia_unknown'])

# Save the cleaned DataFrame to Excel
measurements_standard_clean.to_excel(os.path.join(path_to_measurements_standard, 'measurements_standard_clean.xlsx'), index=False)
# Display the cleaned DataFrame
measurements_standard_clean

# """
# Pivoting this data frame is quite challenging, as some patients have over 1000 measurements. 
# At the moment, simple pivoting would not make the database more human-readable, 
# so alternative approaches such as subsampling or well thought-through filtering must be considered
# in order to achieve good human readability.
# """

,patient_id,measurement_date,weight_kg,bmi,bmr_kcal,fat_%,vat_%,muscle_%,water_%
155,9896A74CEB969,2020-04-03 19:53:22,120.40,35.18,2387.61,44.55,14.45,29.35,35.45
156,9896A74CEB969,2020-04-03 19:54:06,120.40,35.18,2365.45,45.80,14.45,28.22,34.20
192,9896A74CEB969,2020-04-05 19:44:18,116.90,34.16,2345.03,42.25,13.85,30.83,37.75
43892,9896A74CEB969,2021-03-24 17:40:06,128.40,37.52,2483.73,49.80,15.80,25.83,30.20
129197,9896A74CEB969,2022-01-04 15:55:53,127.70,37.72,2468.71,49.78,15.92,25.91,30.22
...,...,...,...,...,...,...,...,...,...
295967,9AA7F84CEB969,2024-02-29 06:54:22,26.29,41.33,5.14,1.00,3.84,1649.38,31.64
296041,9AA8264CEB969,2024-02-29 08:04:41,26.25,42.41,5.13,1.00,4.04,1531.97,33.32
295598,9AA8294CEB969,2024-02-27 08:30:57,33.30,41.25,13.39,1.00,3.84,1955.38,30.44
295909,9AA8314CEB969,2024-02-28 14:28:41,27.87,40.23,5.44,1.00,3.89,1656.38,33.73


SEPARATE CLEANING OF MEASUREMENTS DATA TO HAVE THE FOLLOWING HIERARCHY: 
- measurements_standard: all variable names translated to English
    - measurements_standard_clean: all irrelevant variables (columns) dropped
        - measurements_rowclean: all rows containing outliers dropped
            - measurements_rowclean_longitudinals: all rows representing measurements that is more than 2 months apart from any other measurement dropped
                - measurements_rowclean_deltas: all except the first and last measurements of intervention blocks dropped, and data frame pivoted to wide format

IMPORTANT: GROUPING BY TWO MONTHS MIGHT BE TOO LITTLE AND NEEDS TO BE REVISED!!

UPDATE DESCRIPTION: CAN BE DONE OFFLINE!!
ALSO: SAVE THE DATA FRAMES THAT ARE BEING CREATED? MAYBE YES, MAYBE TOO EARLY TO COMMIT TO THAT? 
Pipeline to further clean measurements: remove outliers and patients that do not have at least two consequent measurements. 

As a result of this pipeline: 
- we start with: measurements_standard_clean with 14 170 patient IDs and 296 129 observations
    - where many observations contain weight values of under 50kg, and body composition values that are off
    - some patients have only one observation, or have isolated observations that are more than 2 months apart from any other observation
    - and there are duplicate observations from the same day when patients step on the scale twice in a row
- we end with: --- with 8311 patient IDs and 169 369 observations
    - where all observed weights are at least 52kg
    - patients have at least two observations within 2 months time
    - some (around 1450) patients have multiple observation groups belonging to multiple intervention periods
    - and there are no duplicate observations from the same day

In [3]:
from datetime import timedelta

# Step 1: Clean rows (remove duplicates, filter outliers, and sort)
def clean_rows(measurements_standard_clean):
    # Remove time info from measurement_date and drop duplicates
    measurements_standard_clean['measurement_date'] = pd.to_datetime(measurements_standard_clean['measurement_date']).dt.date
    
    # Remove rows where patient_id, measurement_date, weight_kg, and bmi are all duplicates
    measurements_standard_clean = measurements_standard_clean.drop_duplicates(
        subset=["patient_id", "measurement_date", "weight_kg", "bmi"]
    )
    
    # Filter rows where weight_kg is 50 or above
    measurements_standard_clean = measurements_standard_clean[measurements_standard_clean['weight_kg'] >= 50]
    
    # Sort by patient_id and measurement_date
    measurements_standard_clean = measurements_standard_clean.sort_values(
        by=["patient_id", "measurement_date"]
    )
    
    return measurements_standard_clean

# Step 2: Filter longitudinal data (remove patients with no continuous block of measurements)
def filter_longitudinal(measurements_standard_clean):
    # Group rows based on patient_id and measurement_date logic
    def group_logic(df):
        # Add time back temporarily for grouping logic
        df["measurement_date"] = pd.to_datetime(df["measurement_date"])
        groups = []
        current_group = []
        for i, row in df.iterrows():
            if not current_group:
                current_group.append(row)
            else:
                last_date = current_group[-1]["measurement_date"]
                if row["measurement_date"] - last_date <= timedelta(days=60):
                    current_group.append(row)
                else:
                    groups.append(current_group)
                    current_group = [row]
        if current_group:
            groups.append(current_group)
        return groups
    
    # Apply grouping logic with explicit patient_id
    grouped_data = []
    for patient_id, group in measurements_standard_clean.groupby("patient_id"):
        # Apply group_logic to each patient's group
        patient_groups = group_logic(group.reset_index(drop=True))
        for g in patient_groups:
            if len(g) > 1:  # Only keep groups with more than one row
                grouped_data.extend(g)
    
    # Convert back to DataFrame
    measurements_standard_clean = pd.DataFrame(grouped_data)
    
    # Remove time info from measurement_date at the end
    measurements_standard_clean['measurement_date'] = measurements_standard_clean['measurement_date'].dt.date
    
    return measurements_standard_clean

# Step 3: Use the modular functions
# Clean rows and save the intermediate output
measurements_rowclean = clean_rows(measurements_standard_clean.copy())
# Save the cleaned rows for cross-sectional analysis
# measurements_rowclean.to_excel("measurements_rowclean.xlsx", index=False)

# Filter longitudinal data and save the final output
measurements_rowclean_longitudinals = filter_longitudinal(measurements_rowclean.copy())
# Save the longitudinal data for further analysis
# measurements_rowclean_long.to_excel("measurements_rowclean_long.xlsx", index=False)

This code: creates the pivoted df with the deltas and intervention durations, as well as noting whether the patient is in the first or a subsequent cycle of intervention. 

IT STILL NEEDS A LOT OF WRANGLING THOUGH!!
- 2 months for an intervention might not be a good cutoff
- some interventions last like a day or a week
- some patients have up to 5-6 interventions, and some of those are short, so that needs to be revised
- and overall i still feel this is a good direction of closing in on the data we are going to actually be able to use. 

In [4]:
import pandas as pd
from datetime import timedelta

def compute_intervention_deltas(measurements):
    """
    Computes intervention blocks for each patient and pivots data to wide-format.
    
    An intervention (or intervention block) is defined as a continuous series of measurements 
    for a patient where consecutive measurement_date values are no more than 60 days apart.
    
    For each block, it calculates:
      - intervention_number (sequential per patient)
      - intervention_start_date: date of the first measurement in the block
      - intervention_end_date: date of the last measurement in the block
      - intervention_duration: difference in days between end and start
      - For each body composition variable provided (baseline, final, delta)
    
    Input:
      measurements: DataFrame with at least the following columns:
          ['patient_id', 'measurement_date', 'weight_kg', 'bmi', 'bmr_kcal', 'fat_%', 'vat_%', 'muscle_%', 'water_%']
    
    Output:
      A wide-format DataFrame with one row per intervention block.
    """
    # List of variables for which we want baseline, final, and delta values.
    var_list = ['weight_kg', 'bmi', 'bmr_kcal', 'fat_%', 'vat_%', 'muscle_%', 'water_%']
    
    # Ensure the measurement_date is in datetime format.
    measurements = measurements.copy()
    measurements['measurement_date'] = pd.to_datetime(measurements['measurement_date'])
    
    results = []
    
    # Group by patient_id.
    for patient_id, group in measurements.groupby("patient_id"):
        group = group.sort_values("measurement_date")  # sort chronologically
        
        intervention_number = 1
        current_block = []
        
        # Iterate rows within this patient.
        for idx, row in group.iterrows():
            if not current_block:
                # Start a new block.
                current_block.append(row)
            else:
                last_date = current_block[-1]['measurement_date']
                # If the gap is 60 days or less, add to current block.
                if (row['measurement_date'] - last_date) <= timedelta(days=60):
                    current_block.append(row)
                else:
                    # Process current block if it has at least 2 measurements.
                    if len(current_block) > 1:
                        # Record intervention block.
                        rec = process_intervention_block(patient_id, intervention_number, current_block, var_list)
                        results.append(rec)
                        intervention_number += 1
                    # Start a new block with the current row.
                    current_block = [row]
        
        # End-of-group: process the last block.
        if len(current_block) > 1:
            rec = process_intervention_block(patient_id, intervention_number, current_block, var_list)
            results.append(rec)
    
    # Convert results (a list of dictionaries) to a DataFrame.
    wide_df = pd.DataFrame(results)
    # Optionally, convert start and end dates back to date only (drop time):
    for col in ['intervention_start_date', 'intervention_end_date']:
        wide_df[col] = pd.to_datetime(wide_df[col]).dt.date
    return wide_df

def process_intervention_block(patient_id, intervention_number, block, var_list):
    """
    Given a list of rows (as a block) from a single intervention,
    returns a dictionary with the following:
      - patient_id
      - intervention_number
      - intervention_start_date, intervention_end_date, intervention_duration
      - For each variable in var_list:
         baseline_variable, final_variable, delta_variable
    """
    # First and last measurement rows.
    baseline = block[0]
    final = block[-1]
    start_date = baseline['measurement_date']
    end_date = final['measurement_date']
    duration = (end_date - start_date).days
    
    rec = {
        "patient_id": patient_id,
        "intervention_number": intervention_number,
        "intervention_start_date": start_date,
        "intervention_end_date": end_date,
        "intervention_duration": duration,
    }
    
    # For every variable, record baseline, final, and delta.
    for var in var_list:
        rec[f"baseline_{var}"] = baseline[var]
        rec[f"final_{var}"] = final[var]
        rec[f"delta_{var}"] = final[var] - baseline[var]
    
    return rec

measurements_rowclean_deltas = compute_intervention_deltas(measurements_rowclean_longitudinals)
measurements_rowclean_deltas

,patient_id,intervention_number,intervention_start_date,intervention_end_date,intervention_duration,baseline_weight_kg,final_weight_kg,delta_weight_kg,baseline_bmi,final_bmi,...,delta_fat_%,baseline_vat_%,final_vat_%,delta_vat_%,baseline_muscle_%,final_muscle_%,delta_muscle_%,baseline_water_%,final_water_%,delta_water_%
0,9896A74CEB969,1,2020-04-03,2020-04-05,2,120.4,116.9,-3.5,35.18,34.16,...,-2.30,14.45,13.85,-0.60,29.35,30.83,1.48,35.45,37.75,2.30
1,9896AA4CEB969,1,2021-02-13,2021-05-26,102,91.1,80.3,-10.8,31.52,27.79,...,-7.31,12.38,10.22,-2.16,31.63,36.23,4.60,44.87,52.18,7.31
2,9896E04CEB969,1,2022-05-07,2022-05-16,9,79.3,77.8,-1.5,27.44,26.92,...,-0.82,10.00,9.70,-0.30,39.10,39.58,0.48,53.88,54.70,0.82
3,9896E54CEB969,1,2022-05-11,2022-05-20,9,96.7,92.6,-4.1,29.85,28.58,...,-2.57,11.39,10.66,-0.73,34.99,36.62,1.63,47.63,50.20,2.57
4,9896EA4CEB969,1,2021-10-30,2022-05-06,188,110.3,92.6,-17.7,34.81,29.23,...,-11.64,14.26,11.04,-3.22,27.27,34.61,7.34,36.59,48.23,11.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10189,9A5DC94CEB969,1,2022-05-17,2022-05-21,4,89.7,87.4,-2.3,30.32,29.54,...,-1.53,5.92,5.77,-0.15,28.85,29.67,0.82,34.53,36.00,1.47
10190,9A5DE84CEB969,1,2022-05-18,2022-05-20,2,72.0,72.0,0.0,27.78,27.78,...,0.50,5.43,5.43,0.00,32.82,32.24,-0.58,41.03,40.54,-0.49
10191,9A5E094CEB969,1,2022-05-17,2022-05-20,3,74.1,71.4,-2.7,27.89,26.87,...,-2.48,5.44,5.25,-0.19,34.26,36.01,1.75,40.56,42.95,2.39
10192,9A5EA34CEB969,1,2022-05-19,2022-05-20,1,74.1,71.6,-2.5,28.59,27.62,...,-2.06,5.59,5.40,-0.19,30.56,31.91,1.35,39.13,41.11,1.98


### Standardize ale -> alleles

Use consistently formatted variable names in English. 

Omit any variables that are clinically irrelevant. 

Pivot the table so that each patient is represented by a single row. 

Characteristics of the data frame: 
- 2472 patients
- 38 distinct variants in total, unsure how many of those are present in all patients
- value: 0/1/2, suppose those are the number of alleles carried
- allele: the exact SNP carried, two-letter code, missing from 42% of patients
- missing IDs: prescription ID, medical record ID

In [19]:
def standardize_alleles(alleles_df):

    # Apply standardized English column names to alleles_df
    def standardize_column_names(df):
        column_mapping = {
            'Fecha': 'genomics_date',
            'Muestra': 'genomics_sample_id',
            'Labsampleid': 'lab_sample_id',
            'namelab': 'lab_name',
            'Rs': 'rs',
            'Value': 'value',
            'Alelo': 'allele'
        }
        df = df.rename(columns=column_mapping)
        return df
    alleles_df = standardize_column_names(alleles_df)
  
    # Standardize the lab_name column
    def capitalize_lab_name(df):
        df['lab_name'] = df['lab_name'].str.upper()
        return df
    alleles_df = capitalize_lab_name(alleles_df)
    def update_lab_name(alleles_df):
        alleles_df['lab_name'] = alleles_df['lab_name'].replace('CG3', 'CESGEN3')
        return alleles_df
    alleles_df = update_lab_name(alleles_df)
    
    # Reorder columns and sort by genomics_sample_id
    alleles_df = alleles_df[['genomics_sample_id', 'rs', 'value', 'allele', 'genomics_date', 'lab_sample_id', 'lab_name']]
    alleles_df = alleles_df.sort_values(by='genomics_sample_id')

    # Match the patient IDs from patients_standard_clean to the genomics sample IDs
    def add_patient_ids(alleles_df):
        # Order patients_standard_clean by sample_id_from_patients
        patients_merge = patients_standard_clean[['patient_id', 'genomics_sample_id']].sort_values(by='genomics_sample_id')
        # Merge alleles_standard with patients_standard_clean_sorted on genomics_sample_id
        alleles_df = alleles_df.merge(
            patients_merge.rename(columns={'genomics_sample_id': 'sample_id_from_patients'}),
            left_on='genomics_sample_id',
            right_on='sample_id_from_patients',
            how='left'
        )
        # Only keep rows with matching genomics_sample_id
        alleles_df = alleles_df[alleles_df['sample_id_from_patients'].notna()]
        # Check if the columns match
        if alleles_df['genomics_sample_id'].equals(alleles_df['sample_id_from_patients']):
            # Drop the duplicate column if they match
            alleles_df = alleles_df.drop(columns=['sample_id_from_patients'])
        # Reorder columns
        alleles_df = alleles_df[['patient_id', 'genomics_sample_id', 'rs', 'value', 'allele', 'genomics_date', 'lab_sample_id', 'lab_name']]
        return alleles_df
    alleles_df = add_patient_ids(alleles_df)
    return alleles_df

# Create and save a standardized data frame with all variables included
alleles_standard = standardize_alleles(alleles_df.copy())
alleles_standard.to_excel(os.path.join(path_to_alleles_standard, 'alleles_standard.xlsx'), index=False)
alleles_standard

# Omit clinically irrelevant columns
alleles_standard_clean = alleles_standard.drop(columns=['genomics_date', 'lab_sample_id', 'lab_name'])
# Create and save a standardized data frame with only the relevant variables included
alleles_standard_clean.to_excel(os.path.join(path_to_alleles_standard, 'alleles_standard_clean.xlsx'), index=False) 
alleles_standard_clean

# Pivot the dataframe to create columns for alleles
alleles_pivot = alleles_standard_clean.pivot(index=['patient_id', 'genomics_sample_id'], columns='rs', values='allele').add_suffix('_allele')
# Pivot the dataframe to create columns for numeric values
values_pivot = alleles_standard_clean.pivot(index=['patient_id', 'genomics_sample_id'], columns='rs', values='value').add_suffix('_value')
# Combine both pivoted data frames
alleles_clean_pivoted = pd.concat([alleles_pivot, values_pivot], axis=1).reset_index()
# Save the pivoted data frame
alleles_clean_pivoted.to_excel(os.path.join(path_to_alleles_standard, 'alleles_clean_pivoted.xlsx'), index=False)
alleles_clean_pivoted

rs,patient_id,genomics_sample_id,CCH_allele,CLK_allele,CLOCK_allele,COACH_allele,INFL_allele,INFLAM_allele,NUT_allele,NUTC_allele,...,rs1801260_value,rs1801282_value,rs361525_value,rs4994_value,rs5082_value,rs601338_value,rs6265_value,rs696217_value,rs7498665_value,rs894160_value
0,9896F54CEB969,68EB8FB3EB8621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9897104CEB969,68F7E3C3EB8621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,98974C4CEB969,68F7C023EB8621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0
3,98976A4CEB969,68EF68F3EB8621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,9897B44CEB969,68FB7733EB8621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,9A9CF74CEB969,68FCE723EB8621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
2468,9A9D254CEB969,68F6EB63EB8621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2469,9A9D934CEB969,68F6C373EB8621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
2470,9A9DF64CEB969,68F71EC3EB8621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Load the standardized and cleaned filed into 2 distinct SQL databases

In [ ]:
import os
import sqlite3

# Define the directory where the SQLite databases will be saved
sqlite_directory = r"C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\DB2_standard"
os.makedirs(sqlite_directory, exist_ok=True)  # Create the directory if it doesn't exist

# Define the paths for the SQLite database files
standard_db_path = os.path.join(sqlite_directory, "pnk_db2_standard.sqlite")
clean_db_path = os.path.join(sqlite_directory, "pnk_db2_standard_clean.sqlite")

"""STANDARD DATABASE"""

# Create a connection to the standard SQLite database
conn = sqlite3.connect(standard_db_path)

# Load each DataFrame into the SQL database as a separate table
patients_standard.to_sql("patients_standard", conn, if_exists="replace", index=False)
medical_records_standard.to_sql("medical_records_standard", conn, if_exists="replace", index=False)
prescriptions_standard.to_sql("prescriptions_standard", conn, if_exists="replace", index=False)
measurements_standard.to_sql("measurements_standard", conn, if_exists="replace", index=False)
alleles_standard.to_sql("alleles_standard", conn, if_exists="replace", index=False)

conn.commit()
conn.close()

print(f"PNK DB2 Standard database created and saved to {standard_db_path}")

"""STANDARD, CLEAN DATABASE"""

# Create a connection to the clean SQLite database
conn = sqlite3.connect(clean_db_path)

# Load each DataFrame into the SQL database as a separate table
patients_standard_clean.to_sql("patients_standard_clean", conn, if_exists="replace", index=False)
medical_records_standard_clean.to_sql("medical_records_standard_clean", conn, if_exists="replace", index=False)
prescriptions_standard_clean.to_sql("prescriptions_standard_clean", conn, if_exists="replace", index=False)
measurements_standard_clean.to_sql("measurements_standard_clean", conn, if_exists="replace", index=False)
alleles_standard_clean.to_sql("alleles_standard_clean", conn, if_exists="replace", index=False)

conn.commit()
conn.close()

print(f"PNK DB2 Standard, clean database created and saved to {clean_db_path}")

## Access all files as Pandas data frames and SQL tables

### Import standardized files

As the standardized files are cross-referenced during the standardization process, once they are created, they should be called externally instead of recreating them every time. Recreating existing files will apparently add duplicate rows to existing ones instead of recreating them from zero. 

In [1]:
import pandas as pd

"""
IMPORT STANDARDIZED FILES WITH ALL AVAILABLE VARIABLES
"""
# path_to_patients_standard = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/patients_standard/patients_standard.xlsx"
# path_to_medical_records_standard = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/medical_records_standard/medical_records_standard.xlsx"
# path_to_prescriptions_standard = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/prescriptions_standard/prescriptions_standard.xlsx"
# path_to_measurements_standard = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/measurements_standard/measurements_standard.xlsx"
# path_to_alleles_standard = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/alleles_standard/alleles_standard.xlsx"

# # Read the excels to pandas data frames
# patients_standard = pd.read_excel(path_to_patients_standard)
# medical_records_standard = pd.read_excel(path_to_medical_records_standard)
# prescriptions_standard = pd.read_excel(path_to_prescriptions_standard)
# measurements_standard = pd.read_excel(path_to_measurements_standard)
# alleles_standard = pd.read_excel(path_to_alleles_standard)

"""
IMPORT STANDARDIZED, CLEANED FILES WITH ONLY THE RELEVANT VARIABLES
"""
path_to_patients_standard_clean = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/patients_standard/patients_standard_clean.xlsx"
path_to_medical_records_standard_clean = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/medical_records_standard/medical_records_standard_clean.xlsx"
path_to_prescriptions_standard_clean = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/prescriptions_standard/prescriptions_standard_clean.xlsx"
path_to_measurements_standard_clean = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/measurements_standard/measurements_standard_clean.xlsx"
path_to_alleles_standard_clean = "C:/Users/Felhasználó/Desktop/Projects/PNK_DB2/DB2_standard/alleles_standard/alleles_standard_clean.xlsx"

# Read the excels to pandas data frames
patients_standard_clean = pd.read_excel(path_to_patients_standard_clean)
medical_records_standard_clean = pd.read_excel(path_to_medical_records_standard_clean)
prescriptions_standard_clean = pd.read_excel(path_to_prescriptions_standard_clean)
measurements_standard_clean = pd.read_excel(path_to_measurements_standard_clean)
alleles_standard_clean = pd.read_excel(path_to_alleles_standard_clean)

Open any given preexisting data frame in Data Wrangler by typing its name to the cell below

In [ ]:
alleles_standard

,medical_record_id,medical_record_creation_date,medical_record_closing_date,birth_date,age,age_when_creating_record,height_m,wc_cm_confirm_time,pnk_method,orders_in_medical_record,...,physical_inactivity_cause,weight_gain_cause,smoking,medications,emotional_eating,satiety,hunger,emotional_eating_value,quantity_control,impulse_control
0,1001F742A682D,2019-02-13 18:38:52,2023-04-28 05:02:45,1977-07-10,45.0,41,1.72,NaN,PNK,46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10032242A682D,2019-02-13 18:40:15,2022-08-13 18:34:03,1961-09-06,59.0,57,1.61,NaN,PNK,8,...,Lesión,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10080342A682D,2019-02-13 18:46:16,2022-08-26 18:02:03,1967-01-21,55.0,52,1.63,NaN,PNK,28,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100DAF42A682D,2019-02-13 18:53:16,2022-07-16 18:06:03,1980-10-17,41.0,38,1.58,NaN,PNK,30,...,Falta de tiempo,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100E3742A682D,2019-02-13 18:53:57,2021-01-07 21:05:36,1978-06-09,43.0,40,1.72,NaN,PNK,1,...,Lesión,"RITMO DE TRABALHO MUITO STRESSANTE, COMIDA",No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20937,FF2B242A682D,2019-02-13 18:16:04,2021-02-16 21:07:13,1974-07-26,48.0,44,1.56,NaN,PNK,2,...,NaN,QUANDO INICIOU FACULDADE PAROU DE FAZER EXERCI...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20938,FF86B42A682D,2019-02-13 18:23:15,NaT,1957-05-19,66.0,61,1.56,NaN,PNK EXPERT,92,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20939,FFAD342A682D,2019-02-13 18:26:19,2021-03-04 21:33:46,1986-11-22,34.0,32,1.60,NaN,PNK,2,...,NaN,MUDOU-SE PARA LISBOA - TRABALHO MAIS SEDENTARI...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20940,FFAE442A682D,2019-02-13 18:26:24,2021-07-08 21:35:06,1953-07-10,68.0,65,1.48,NaN,PNK,8,...,NaN,HISTERECTOMIA EM 1990 E NAO CONSEGUE MANTER O ...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Address additional data cleaning issues

### See how many of the 'well-measured' patients have alleles and eating behavior data

THIS ALL COULD USE SOME NICE DOCUMENTATION SOON!!

In [ ]:
"""
See the WEIGHT EVOLUTION of patients with genomics data available.
"""

# Get the unique patient IDs from the alleles data
allele_patient_ids = alleles_standard_clean['patient_id'].unique()

# Subset the measurements data to include only those patients who are in the alleles data
measurements_standard_clean_alleles = measurements_standard_clean[
    measurements_standard_clean['patient_id'].isin(allele_patient_ids)
]
measurements_standard_clean_alleles

"""
See how many of the patients with RELIABLE WEIGHT EVOLUTION data have genomics data available: 1413 patients.
"""

# Get the unique patient IDs from the alleles data
allele_patient_ids = alleles_standard_clean['patient_id'].unique()

# Subset the measurements data to include only those patients who are in the alleles data
measurements_rowclean_alleles = measurements_rowclean_longitudinals[
    measurements_rowclean_longitudinals['patient_id'].isin(allele_patient_ids)
]
measurements_rowclean_alleles

"""
See the WEIGHT EVOLUTION data of patients with EATING BEHAVIOR data available.
"""

def get_eating_behavior_available(medical_records):
    """
    Returns the subset of medical_records where the following columns are not NA:
    'emotional_eating', 'satiety', 'hunger', 'emotional_eating_value',
    'quantity_control', 'impulse_control'
    """
    required_cols = [
        'emotional_eating',
        'satiety',
        'hunger',
        'emotional_eating_value',
        'quantity_control',
        'impulse_control'
    ]
    return medical_records.dropna(subset=required_cols)

# Example usage:
# Get the subset of patients with eating behavior data available.
eating_behavior_available = get_eating_behavior_available(medical_records_standard_clean)

# Now, based on these patients, subset the longitudinal measurements.
eating_behavior_ids = eating_behavior_available['patient_id'].unique()

measurements_standard_clean_eating_behavior = measurements_standard_clean[
    measurements_standard_clean['patient_id'].isin(eating_behavior_ids)
]
# Display the subset
measurements_standard_clean_eating_behavior

"""
See how many of the patients with RELIABLE WEIGHT EVOLUTION data have EATING BEHAVIOR data available: 556 patients.
"""

measurements_longitudinal_eating_behavior = measurements_rowclean_longitudinals[
    measurements_rowclean_longitudinals['patient_id'].isin(eating_behavior_ids)
]
# Display the subset
measurements_longitudinal_eating_behavior

"""
See how many of the patients with RELIABLE WEIGHT EVOLUTION data have both GENOMICS AND EATING BEHAVIOR data available: 118 patients.
"""

# Step 1: Get common patient IDs between the alleles and eating behavior measurements
alleles_ids = set(measurements_rowclean_alleles['patient_id'].unique())
eating_ids = set(measurements_longitudinal_eating_behavior['patient_id'].unique())
common_ids = alleles_ids.intersection(eating_ids)

# Step 2: Subset the measurements_rowclean_longitudinals data frame for only those patients
measurements_rowclean_alleles_eating_behavior = measurements_rowclean_longitudinals[
    measurements_rowclean_longitudinals['patient_id'].isin(common_ids)
]

# Display the resulting DataFrame
measurements_rowclean_alleles_eating_behavior


,patient_id,measurement_date,weight_kg,bmi,bmr_kcal,fat_%,vat_%,muscle_%,water_%
0,98976A4CEB969,2021-07-06,72.8,26.74,1514.02,35.71,5.23,33.54,42.52
1,98976A4CEB969,2021-07-24,71.8,26.37,1477.03,35.85,5.16,32.98,42.39
2,98976A4CEB969,2021-08-01,71.6,26.30,1491.95,35.04,5.14,33.82,43.16
3,98976A4CEB969,2021-08-02,71.3,26.19,1466.50,35.62,5.12,33.04,42.61
4,98976A4CEB969,2021-08-06,70.9,26.04,1472.99,34.87,5.09,33.73,43.32
...,...,...,...,...,...,...,...,...,...
1,9A5BDF4CEB969,2022-05-19,93.6,27.95,1859.00,42.29,5.47,26.36,36.21
0,9A5CA54CEB969,2022-05-11,81.3,26.85,1757.83,26.52,9.67,38.03,53.48
1,9A5CA54CEB969,2022-05-17,80.5,26.59,1760.34,25.30,9.51,38.99,54.70
0,9A5D564CEB969,2022-05-13,74.0,26.85,1489.81,37.32,5.26,30.42,40.97


Explore associations between self-reported eating behavior and weight loss success. 

In [ ]:
measurements_longitudinal_eating_behavior

In [37]:
import pandas as pd
from datetime import timedelta

def compute_intervention_deltas(measurements):
    """
    Computes intervention blocks for each patient and pivots data to wide-format.
    
    An intervention (or intervention block) is defined as a continuous series of measurements 
    for a patient where consecutive measurement_date values are no more than 90 days apart.
    
    For each block, it calculates:
      - treatment_cycle (sequential per patient)
      - intervention_start_date: date of the first measurement in the block
      - intervention_end_date: date of the last measurement in the block
      - intervention_duration: difference in days between end and start
      - For each body composition variable provided (baseline, final, delta)
    
    Input:
      measurements: DataFrame with at least the following columns:
          ['patient_id', 'measurement_date', 'weight_kg', 'bmi', 'bmr_kcal', 'fat_%', 'vat_%', 'muscle_%', 'water_%']
    
    Output:
      A wide-format DataFrame with one row per intervention block.
    """
    # List of variables for which we want baseline, final, and delta values.
    var_list = ['weight_kg', 'bmi', 'bmr_kcal', 'fat_%', 'vat_%', 'muscle_%', 'water_%']
    
    # Ensure the measurement_date is in datetime format.
    measurements = measurements.copy()
    measurements['measurement_date'] = pd.to_datetime(measurements['measurement_date'])
    
    results = []
    
    # Group by patient_id.
    for patient_id, group in measurements.groupby("patient_id"):
        group = group.sort_values("measurement_date")  # sort chronologically
        
        treatment_cycle = 1
        current_block = []
        
        # Iterate rows within this patient.
        for idx, row in group.iterrows():
            if not current_block:
                # Start a new block.
                current_block.append(row)
            else:
                last_date = current_block[-1]['measurement_date']
                # If the gap is 60 days or less, add to current block.
                if (row['measurement_date'] - last_date) <= timedelta(days=90):
                    current_block.append(row)
                else:
                    # Process current block if it has at least 2 measurements.
                    if len(current_block) > 1:
                        # Record intervention block.
                        rec = process_intervention_block(patient_id, treatment_cycle, current_block, var_list)
                        results.append(rec)
                        treatment_cycle += 1
                    # Start a new block with the current row.
                    current_block = [row]
        
        # End-of-group: process the last block.
        if len(current_block) > 1:
            rec = process_intervention_block(patient_id, treatment_cycle, current_block, var_list)
            results.append(rec)
    
    # Convert results (a list of dictionaries) to a DataFrame.
    wide_df = pd.DataFrame(results)
    # Optionally, convert start and end dates back to date only (drop time):
    for col in ['intervention_start_date', 'intervention_end_date']:
        wide_df[col] = pd.to_datetime(wide_df[col]).dt.date
    return wide_df

def process_intervention_block(patient_id, treatment_cycle, block, var_list):
    """
    Given a list of rows (as a block) from a single intervention,
    returns a dictionary with the following:
      - patient_id
      - treatment_cycle
      - intervention_start_date, intervention_end_date, intervention_duration
      - For each variable in var_list:
         baseline_variable, final_variable, delta_variable
    """
    # First and last measurement rows.
    baseline = block[0]
    final = block[-1]
    start_date = baseline['measurement_date']
    end_date = final['measurement_date']
    duration = (end_date - start_date).days
    
    rec = {
        "patient_id": patient_id,
        "treatment_cycle": treatment_cycle,
        "intervention_start_date": start_date,
        "intervention_end_date": end_date,
        "intervention_duration": duration,
    }
    
    # For every variable, record baseline, final, and delta.
    for var in var_list:
        rec[f"baseline_{var}"] = baseline[var]
        rec[f"final_{var}"] = final[var]
        rec[f"delta_{var}"] = final[var] - baseline[var]
    
    return rec

measurements_emotional__baselinefinal_deltas = compute_intervention_deltas(measurements_longitudinal_eating_behavior)
measurements_emotional__baselinefinal_deltas

,patient_id,treatment_cycle,intervention_start_date,intervention_end_date,intervention_duration,baseline_weight_kg,final_weight_kg,delta_weight_kg,baseline_bmi,final_bmi,...,delta_fat_%,baseline_vat_%,final_vat_%,delta_vat_%,baseline_muscle_%,final_muscle_%,delta_muscle_%,baseline_water_%,final_water_%,delta_water_%
0,9896E54CEB969,1,2022-05-11,2022-05-20,9,96.7,92.6,-4.1,29.85,28.58,...,-2.57,11.39,10.66,-0.73,34.99,36.62,1.63,47.63,50.20,2.57
1,9897484CEB969,1,2022-04-24,2022-05-04,10,63.9,60.5,-3.4,26.60,25.18,...,-2.30,5.20,4.93,-0.27,34.77,36.03,1.26,43.43,45.64,2.21
2,98976A4CEB969,1,2021-07-06,2021-10-01,87,72.8,70.5,-2.3,26.74,25.90,...,-1.38,5.23,5.06,-0.17,33.54,34.19,0.65,42.52,43.84,1.32
3,9898914CEB969,1,2020-08-05,2020-09-16,42,67.3,62.0,-5.3,27.30,25.15,...,-3.45,5.34,4.92,-0.42,33.64,35.44,1.80,42.15,45.46,3.31
4,9898914CEB969,2,2021-06-19,2021-07-11,22,64.7,62.0,-2.7,26.59,25.48,...,-1.64,5.20,4.99,-0.21,33.30,34.08,0.78,43.05,44.62,1.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,9A5D564CEB969,1,2022-05-13,2022-05-16,3,74.0,74.0,0.0,26.85,26.85,...,0.05,5.26,5.26,0.00,30.42,30.37,-0.05,40.97,40.93,-0.04
679,9A5D604CEB969,1,2022-05-13,2022-05-14,1,79.9,79.0,-0.9,27.32,27.02,...,-0.05,9.91,9.74,-0.17,41.52,41.41,-0.11,54.49,54.54,0.05
680,9A5D684CEB969,1,2022-05-17,2022-05-20,3,89.4,88.4,-1.0,28.86,28.54,...,-0.52,10.79,10.60,-0.19,40.82,41.12,0.30,51.31,51.83,0.52
681,9A5D754CEB969,1,2022-05-17,2022-05-20,3,74.3,72.1,-2.2,27.62,26.81,...,-1.85,5.40,5.24,-0.16,32.26,33.48,1.22,40.41,42.18,1.77


In [38]:
import pandas as pd
from datetime import timedelta

def compute_observation_deltas(measurements):
    """
    Computes observation blocks (treatment cycles) for each patient from the given measurements data 
    and pivots data to wide-format.
    
    An observation block is defined as a continuous series of measurements for a patient where any 
    subsequent measurement is not more than 90 days apart.
    
    For each block, the baseline is the first measurement, and the final measurement is chosen as the 
    measurement closest to 70 days from baseline (but only if it falls within 60 to 80 days). If no measurement 
    is found in that window, the block is skipped.
    
    For each valid block, this function computes:
      - treatment_cycle (sequential per patient)
      - observation_start_date: date of the baseline measurement in the block
      - observation_end_date: date of the measurement selected (~70 days)
      - observation_period_length: days between these dates
      - For each body composition variable in var_list, baseline, final, and delta values
    
    Also returns a DataFrame of excluded observation blocks (e.g., those with no measurement in the desired window).
    
    Input:
      measurements: DataFrame with columns:
         ['patient_id', 'measurement_date', 'weight_kg', 'bmi', 'bmr_kcal', 'fat_%', 'vat_%', 'muscle_%', 'water_%']
    
    Output:
      wide_df: A wide-format DataFrame with one row per valid observation block.
      excluded_df: A DataFrame listing the observation blocks skipped, with patient_id, treatment_cycle, and reason.
    """
    # List of body composition variables.
    var_list = ['weight_kg', 'bmi', 'bmr_kcal', 'fat_%', 'vat_%', 'muscle_%', 'water_%']
    
    # Make sure measurement_date is datetime.
    measurements = measurements.copy()
    measurements['measurement_date'] = pd.to_datetime(measurements['measurement_date'])
    
    results = []
    excluded = []  # To record skipped blocks
    
    # Group by patient.
    for patient_id, group in measurements.groupby("patient_id"):
        group = group.sort_values("measurement_date")  # sort chronologically
        treatment_cycle = 1
        current_block = []
        
        # Iterate over measurements for the patient.
        for idx, row in group.iterrows():
            if not current_block:
                current_block.append(row)
            else:
                last_date = current_block[-1]['measurement_date']
                # A gap of 90 days or less continues current block.
                if (row['measurement_date'] - last_date) <= timedelta(days=90):
                    current_block.append(row)
                else:
                    # Process the current block.
                    rec = process_observation_block(patient_id, treatment_cycle, current_block, var_list)
                    if rec is None:
                        excluded.append({
                            'patient_id': patient_id,
                            'treatment_cycle': treatment_cycle,
                            'reason': 'No measurement in 60-80 days window',
                            'observation_start_date': current_block[0]['measurement_date']
                        })
                    else:
                        results.append(rec)
                    treatment_cycle += 1
                    current_block = [row]
        
        # Process the final block for the patient.
        if current_block:
            rec = process_observation_block(patient_id, treatment_cycle, current_block, var_list)
            if rec is None:
                excluded.append({
                    'patient_id': patient_id,
                    'treatment_cycle': treatment_cycle,
                    'reason': 'No measurement in 60-80 days window',
                    'observation_start_date': current_block[0]['measurement_date']
                })
            else:
                results.append(rec)
    
    wide_df = pd.DataFrame(results)
    # Convert observation date columns to date (drop time info)
    for col in ['observation_start_date', 'observation_end_date']:
        wide_df[col] = pd.to_datetime(wide_df[col]).dt.date
    excluded_df = pd.DataFrame(excluded)
    return wide_df, excluded_df

def process_observation_block(patient_id, treatment_cycle, block, var_list):
    """
    Processes a block of measurements (an observation block) for a single treatment cycle.
    
    Baseline is the first measurement in block.
    It then selects the measurement that occurs between 60 and 80 days after baseline
    and is closest to 70 days. If no such measurement exists, returns None.
    
    Returns a dictionary with:
      - patient_id, treatment_cycle, observation_start_date, 
        observation_end_date, observation_period_length, and for each variable:
         baseline_variable, final_variable, delta_variable.
    """
    baseline = block[0]
    baseline_date = baseline['measurement_date']
    
    # Find measurements where days from baseline are between 60 and 80.
    candidate_rows = [row for row in block if 40 <= (row['measurement_date'] - baseline_date).days <= 80]
    
    if not candidate_rows:
        return None  # Block does not provide a proper 70-day measurement.
    
    # Choose candidate closest to 70 days.
    candidate = min(candidate_rows, key=lambda row: abs((row['measurement_date'] - baseline_date).days - 60))
    
    observation_start_date = baseline_date
    observation_end_date = candidate['measurement_date']
    period_length = (observation_end_date - observation_start_date).days
    
    rec = {
        "patient_id": patient_id,
        "treatment_cycle": treatment_cycle,
        "observation_start_date": observation_start_date,
        "observation_end_date": observation_end_date,
        "observation_period_length": period_length,
    }
    
    for var in var_list:
        rec[f"baseline_{var}"] = baseline[var]
        rec[f"final_{var}"] = candidate[var]
        rec[f"delta_{var}"] = candidate[var] - baseline[var]
    
    return rec

measurements_emotional_60day_deltas, excluded_obs_df = compute_observation_deltas(measurements_longitudinal_eating_behavior.copy())
measurements_emotional_60day_deltas = measurements_emotional_60day_deltas.drop_duplicates(subset=['patient_id'])
print("Valid observation blocks:", measurements_emotional_60day_deltas.shape[0])
print("Excluded blocks:", excluded_obs_df.shape[0])
# excluded_obs_df
measurements_emotional_60day_deltas

Valid observation blocks: 208
Excluded blocks: 451


,patient_id,treatment_cycle,observation_start_date,observation_end_date,observation_period_length,baseline_weight_kg,final_weight_kg,delta_weight_kg,baseline_bmi,final_bmi,...,delta_fat_%,baseline_vat_%,final_vat_%,delta_vat_%,baseline_muscle_%,final_muscle_%,delta_muscle_%,baseline_water_%,final_water_%,delta_water_%
0,98976A4CEB969,1,2021-07-06,2021-09-13,69,72.8,70.5,-2.3,26.74,25.90,...,-1.17,5.23,5.06,-0.17,33.54,33.95,0.41,42.52,43.64,1.12
1,9898914CEB969,1,2020-08-05,2020-09-16,42,67.3,62.0,-5.3,27.30,25.15,...,-3.45,5.34,4.92,-0.42,33.64,35.44,1.80,42.15,45.46,3.31
2,9899144CEB969,2,2022-03-02,2022-04-29,58,92.1,83.9,-8.2,31.87,29.03,...,-5.52,6.22,5.67,-0.55,25.85,28.86,3.01,31.33,36.63,5.30
3,989A264CEB969,1,2020-04-21,2020-05-31,40,73.4,66.7,-6.7,27.63,25.10,...,-5.14,5.40,4.92,-0.48,30.11,33.28,3.17,40.04,44.97,4.93
4,989BD04CEB969,1,2020-12-16,2021-02-13,59,88.9,86.5,-2.4,33.46,32.56,...,-1.75,6.53,6.35,-0.18,24.43,25.46,1.03,30.02,31.71,1.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,9A552C4CEB969,1,2022-04-06,2022-05-19,43,82.1,74.4,-7.7,29.09,26.36,...,-5.32,5.68,5.16,-0.52,28.08,31.05,2.97,36.60,41.71,5.11
228,9A55794CEB969,1,2022-04-07,2022-05-17,40,81.1,74.8,-6.3,30.15,27.81,...,-4.70,5.89,5.44,-0.45,26.90,29.74,2.84,35.15,39.67,4.52
229,9A55834CEB969,1,2022-04-05,2022-05-18,43,114.3,102.6,-11.7,38.19,34.28,...,-7.68,16.25,13.99,-2.26,21.68,26.62,4.94,31.76,39.44,7.68
230,9A563A4CEB969,1,2022-04-06,2022-05-17,41,121.4,111.7,-9.7,35.09,32.29,...,-6.08,14.44,12.81,-1.63,24.56,28.40,3.84,34.16,40.24,6.08


In [ ]:
# Merge measurements_emotional_60day_deltas with patients_standard_clean
merged_df = measurements_emotional_60day_deltas.merge(
    patients_standard_clean,
    on='patient_id',
    how='left'
)

# Merge the result with medical_records_standard_clean
merged_df = merged_df.merge(
    medical_records_standard_clean,
    on='patient_id',
    how='left'
)

# Display the merged DataFrame
experiment_321 = merged_df
experiment_321

In [48]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency

# Define the input DataFrame
input_df = experiment_321  # Replace this with the desired input DataFrame

# Continuous variables that have baseline and delta measures.
continuous_vars_baseline = ['weight_kg', 'bmi', 'bmr_kcal', 'fat_%', 'vat_%', 'muscle_%', 'water_%']
# Continuous variables available only as raw values.
continuous_vars_raw = ['impulse_control', 'quantity_control', 'emotional_eating_value']

# List of categorical variables.
categorical_vars = ['hunger', 'satiety', 'emotional_eating']

summary_rows = []

# Process continuous variables with baseline and delta measurements.
for var in continuous_vars_baseline:
    for prefix in ['baseline', 'delta']:
        col_name = f"{prefix}_{var}"
        # Extract data for females and males.
        female_data = input_df.loc[input_df['sex'] == 'M', col_name].dropna()
        male_data   = input_df.loc[input_df['sex'] == 'V', col_name].dropna()
        
        # Calculate mean and sample standard deviation for each group.
        female_mean = np.mean(female_data)
        female_std  = np.std(female_data, ddof=1)
        male_mean   = np.mean(male_data)
        male_std    = np.std(male_data, ddof=1)
        
        # Run an independent t-test (assume unequal variances).
        t_stat, p_val = ttest_ind(female_data, male_data, equal_var=False)
        
        summary_rows.append({
            "variable": col_name,
            "female_avg": female_mean,
            "female_std": female_std,
            "male_avg": male_mean,
            "male_std": male_std,
            "p_value": p_val
        })

# Process continuous variables available as raw values.
for col_name in continuous_vars_raw:
    female_data = input_df.loc[input_df['sex'] == 'M', col_name].dropna()
    male_data   = input_df.loc[input_df['sex'] == 'V', col_name].dropna()
    
    female_mean = np.mean(female_data)
    female_std  = np.std(female_data, ddof=1)
    male_mean   = np.mean(male_data)
    male_std    = np.std(male_data, ddof=1)
    
    t_stat, p_val = ttest_ind(female_data, male_data, equal_var=False)
    
    summary_rows.append({
        "variable": col_name,
        "female_avg": female_mean,
        "female_std": female_std,
        "male_avg": male_mean,
        "male_std": male_std,
        "p_value": p_val
    })

# Process categorical variables.
for var in categorical_vars:
    female_cat = input_df.loc[input_df['sex'] == 'M', var].dropna()
    male_cat   = input_df.loc[input_df['sex'] == 'V', var].dropna()
    
    # Calculate percentage of "Si" responses.
    female_pct = 100 * (female_cat == 'Si').sum() / female_cat.shape[0] if female_cat.shape[0] > 0 else np.nan
    male_pct   = 100 * (male_cat == 'Si').sum() / male_cat.shape[0] if male_cat.shape[0] > 0 else np.nan
    
    # Build the contingency table.
    female_si   = (female_cat == 'Si').sum()
    female_nosi = female_cat.shape[0] - female_si
    male_si     = (male_cat == 'Si').sum()
    male_nosi   = male_cat.shape[0] - male_si
    
    contingency = np.array([[female_si, female_nosi],
                            [male_si,   male_nosi]])
    
    chi2, p_val, dof, expected = chi2_contingency(contingency)
    
    summary_rows.append({
        "variable": var,
        "female_avg": female_pct,
        "female_std": "",
        "male_avg": male_pct,
        "male_std": "",
        "p_value": p_val
    })

# Create the summary table.
comparison_table = pd.DataFrame(summary_rows)

# Calculate the number of females and males and their proportions.
num_females = input_df.loc[input_df['sex'] == 'M'].shape[0]
num_males = input_df.loc[input_df['sex'] == 'V'].shape[0]
total = num_females + num_males
female_pct = (num_females / total) * 100
male_pct = (num_males / total) * 100

# Add the "sex" row to the summary table.
sex_row = {
    "variable": "sex",
    "female_avg": num_females,
    "female_std": f"{female_pct:.2f}%",
    "male_avg": num_males,
    "male_std": f"{male_pct:.2f}%",
    "p_value": ""
}
comparison_table = pd.concat([pd.DataFrame([sex_row]), comparison_table], ignore_index=True)

# Round numeric columns to the desired precision.
comparison_table['female_avg'] = pd.to_numeric(comparison_table['female_avg'], errors='coerce').round(4)
comparison_table['female_std'] = pd.to_numeric(comparison_table['female_std'], errors='coerce').round(4)
comparison_table['male_avg'] = pd.to_numeric(comparison_table['male_avg'], errors='coerce').round(4)
comparison_table['male_std'] = pd.to_numeric(comparison_table['male_std'], errors='coerce').round(4)

# Do not round p-values.
comparison_table['p_value'] = pd.to_numeric(comparison_table['p_value'], errors='coerce')

# Display the table.
print(comparison_table)

                  variable  female_avg  female_std   male_avg  male_std  \
0                      sex    436.0000         NaN   101.0000       NaN   
1       baseline_weight_kg     82.4390      7.9229   101.1356   15.7774   
2          delta_weight_kg     -7.3048      3.6598   -11.0485    5.7394   
3             baseline_bmi     30.7406      2.2878    32.6506    4.2626   
4                delta_bmi     -2.7570      1.4134    -3.5723    1.8355   
5        baseline_bmr_kcal   1626.0764    143.1855  2034.0714  250.0510   
6           delta_bmr_kcal   -107.6994     56.0013  -142.9020   83.8551   
7           baseline_fat_%     43.5684      4.2063    38.4290    7.9894   
8              delta_fat_%     -4.9865      2.6390    -6.5578    4.5718   
9           baseline_vat_%      6.1013      1.1489    12.7043    3.0502   
10             delta_vat_%     -0.6278      0.9628    -2.2548    1.8956   
11       baseline_muscle_%     28.1753      2.9048    31.4782    5.0420   
12          delta_muscle_

----S-E-P-E-R-E-Y-T-A----

In [ ]:
patients_standard_clean

In [ ]:
medical_records_standard_clean

In [ ]:
prescriptions_standard_clean

In [ ]:
alleles_standard_clean

In [ ]:
measurements_standard_clean

In [ ]:
measurements_rowclean

In [ ]:
measurements_rowclean_longitudinals

In [ ]:
measurements_rowclean_deltas

In [ ]:
measurements_standard_clean_alleles

In [ ]:
measurements_rowclean_alleles

In [ ]:
measurements_standard_clean_eating_behavior

In [ ]:
measurements_rowclean_eating_behavior

In [ ]:
measurements_rowclean_alleles_eating_behavior

## Query the SQL database about whatever you want to know

### General queries

Build an 'ID DB', a data frame that contains all the relevant identifiers with their correspondences (patient, medical record, prescription and genomics sample IDs).

In [35]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('pnk_db2_standard_clean.sqlite')

# Build the query
query = """
WITH prescription_ids AS (
    -- First get unique combinations from prescriptions table
    SELECT DISTINCT
        patient_id,
        prescription_id,
        medical_record_id
    FROM prescriptions_standard_clean
),
genomics_ids AS (
    -- Get genomics sample IDs for patients
    SELECT DISTINCT
        patient_id,
        genomics_sample_id
    FROM patients_standard_clean
    WHERE genomics_sample_id IS NOT NULL
)
-- Combine prescriptions and genomics data
SELECT 
    p.*,
    g.genomics_sample_id
FROM prescription_ids p
LEFT JOIN genomics_ids g ON p.patient_id = g.patient_id
"""

# Create the ID database
id_db = pd.read_sql_query(query, conn)

# Print summary statistics
print("ID Database Summary:")
print("-" * 50)
print(f"Total number of records: {len(id_db)}")
print(f"Number of unique patients: {id_db['patient_id'].nunique()}")
print(f"Number of unique medical records: {id_db['medical_record_id'].nunique()}")
print(f"Number of unique prescriptions: {id_db['prescription_id'].nunique()}")
print(f"Number of unique genomics samples: {id_db['genomics_sample_id'].nunique()}")

# Check for completeness
print("\nCompleteness Check:")
print("-" * 50)
print(id_db.info())

# Check for any remaining duplicates
print("\nDuplicate Check:")
print("-" * 50)
duplicate_counts = id_db.groupby('patient_id').size()
if any(duplicate_counts > 1):
    print("Warning: Some patients have multiple records")
    print("\nPatients with multiple records:")
    print(duplicate_counts[duplicate_counts > 1].head())
else:
    print("No duplicates found")

# Display first few rows
print("\nFirst few rows of the ID database:")
print("-" * 50)
display(id_db.head())

# Close the connection
conn.close()

# Optional: Save the DataFrame
# id_db.to_csv('id_database.csv', index=False)

ID Database Summary:
--------------------------------------------------
Total number of records: 113709
Number of unique patients: 14170
Number of unique medical records: 20942
Number of unique prescriptions: 113709
Number of unique genomics samples: 2579

Completeness Check:
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113709 entries, 0 to 113708
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   patient_id          113709 non-null  object
 1   prescription_id     113709 non-null  object
 2   medical_record_id   113709 non-null  object
 3   genomics_sample_id  31422 non-null   object
dtypes: object(4)
memory usage: 3.5+ MB
None

Duplicate Check:
--------------------------------------------------

Patients with multiple records:
patient_id
9896A74CEB969     5
9896AA4CEB969    23
9896E04CEB969     7
9896EA4CEB969     8
9896F14CEB969     4
dtype: int

,patient_id,prescription_id,medical_record_id,genomics_sample_id
0,9896A74CEB969,146C624D15241,12902F42A682D,None
1,9896A74CEB969,1450B74D15241,12902F42A682D,None
2,9896A74CEB969,12A5604D15241,10ABF442A682D,None
3,9896A74CEB969,1709924D15241,1385C142A682D,None
4,9896A74CEB969,16C8CA4D15241,1385C142A682D,None


Query the list of relevant and more or less complete variables, contained in the standard_clean database

In [50]:
# List all the variables in all the tables in the clean database

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("pnk_db2_standard_clean.sqlite")

# Query to get all table names
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# List all columns for each table
for table in tables:
    table_name = table[0]
    print(f"Table: {table_name}")
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    for column in columns:
        print(f"  - {column[1]}")
    print()

# Close the connection
conn.close()

Table: medical_record_standard_clean
  - medical_record_id
  - medical_record_creation_date
  - medical_record_closing_date
  - birth_date
  - age
  - age_when_creating_record
  - height_m
  - wc_cm_confirm_time
  - pnk_method
  - orders_in_medical_record
  - dietitian_visits
  - physical_activity
  - physical_activity_frequency
  - physical_inactivity_cause
  - weight_gain_cause
  - smoking
  - medications
  - emotional_eating
  - satiety
  - hunger
  - emotional_eating_value
  - quantity_control
  - impulse_control

Table: prescription_standard_clean
  - patient_id
  - prescription_id
  - medical_record_id
  - prescription_creation_date
  - prescription_registration_date
  - prescription_validity_end_date
  - prescription_validity_days
  - method
  - step
  - consumed_meals
  - pending_meals
  - daily_meals
  - total_meals
  - purchased_meals_<_prescribed
  - cumulative_expenses
  - genomics
  - genomics_prescription_date
  - pack_123_pnk
  - pack_123_pnkexpert
  - pack_123_pnkexpert

### Eating behavior with genetics-related queries

Query the available SNPs, and the number of patients they are collected from. 

In [51]:
import sqlite3

# Connect to the SQLite database file
conn = sqlite3.connect('pnk_db2_standard_clean.sqlite')
cursor = conn.cursor()

# Count total number of distinct patients
cursor.execute("SELECT COUNT(DISTINCT patient_id) FROM alleles_standard_clean")
total_patients = cursor.fetchone()[0]

# Query to find SNPs present in all patients
query = """
SELECT rs, COUNT(DISTINCT patient_id) as patient_count
FROM alleles_standard_clean
WHERE value IS NOT NULL
GROUP BY rs
HAVING COUNT(DISTINCT patient_id) = ?
ORDER BY rs;
"""

cursor.execute(query, (total_patients,))
results = cursor.fetchall()

print(f"Total number of patients: {total_patients}")
print("\nSNPs present in all patients:")
if len(results) == 0:
    print("No SNPs found in all patients")
else:
    for row in results:
        print(f"SNP: {row[0]} (present in {row[1]} patients)")

# Let's also check the distribution of SNPs
verification_query = """
SELECT rs, COUNT(DISTINCT patient_id) as patient_count
FROM alleles_standard_clean
WHERE value IS NOT NULL
GROUP BY rs
ORDER BY patient_count DESC
LIMIT 38;
"""

print("\nMost common SNPs:")
cursor.execute(verification_query)
results = cursor.fetchall()

# Print results as a numbered list with coverage percentage
for i, row in enumerate(results, 1):
    coverage_percentage = (row[1] / total_patients) * 100
    print(f"{i}. SNP: {row[0]} (present in {row[1]} patients, {coverage_percentage:.1f}% coverage)")

conn.close()

Total number of patients: 2472

SNPs present in all patients:
No SNPs found in all patients

Most common SNPs:
1. SNP: rs894160 (present in 2469 patients, 99.9% coverage)
2. SNP: rs7498665 (present in 2469 patients, 99.9% coverage)
3. SNP: rs696217 (present in 2469 patients, 99.9% coverage)
4. SNP: rs6265 (present in 2469 patients, 99.9% coverage)
5. SNP: rs601338 (present in 2469 patients, 99.9% coverage)
6. SNP: rs5082 (present in 2469 patients, 99.9% coverage)
7. SNP: rs4994 (present in 2469 patients, 99.9% coverage)
8. SNP: rs361525 (present in 2469 patients, 99.9% coverage)
9. SNP: rs1801282 (present in 2469 patients, 99.9% coverage)
10. SNP: rs1801260 (present in 2469 patients, 99.9% coverage)
11. SNP: rs1800896 (present in 2469 patients, 99.9% coverage)
12. SNP: rs1800795 (present in 2469 patients, 99.9% coverage)
13. SNP: rs1800206 (present in 2469 patients, 99.9% coverage)
14. SNP: rs1799883 (present in 2469 patients, 99.9% coverage)
15. SNP: rs17782313 (present in 2469 patien

Identify the subset of patients with available data on eating behavior in the medical_records_standard_clean registry - 1499 patients if excluding duplicates (some patients, in the number of the thousands, have at least two medical records. This might be due to diet restart, and it needs to be verified).

In [7]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('pnk_db2_standard_clean.sqlite')

# Define the columns we want to check for non-NA values
eating_behavior_cols = [
    'emotional_eating',
    'satiety',
    'hunger',
    'emotional_eating_value',
    'quantity_control',
    'impulse_control'
]

# Modified query to select patient_id first
query = f"""
WITH unique_patient_records AS (
    SELECT DISTINCT medical_record_id, patient_id
    FROM prescriptions_standard_clean
)
SELECT p.patient_id, m.*
FROM medical_records_standard_clean m
JOIN unique_patient_records p ON m.medical_record_id = p.medical_record_id
WHERE {' AND '.join([f'm.{col} IS NOT NULL' for col in eating_behavior_cols])}
"""

# Load the data into a pandas DataFrame
eating_behavior_df = pd.read_sql_query(query, conn)

# Remove rows where patient_id appears more than once
eating_behavior_df = eating_behavior_df[~eating_behavior_df['patient_id'].duplicated(keep=False)]

# Sort by patient_id
eating_behavior_df = eating_behavior_df.sort_values('patient_id', ascending=True)

# Print some basic information about the subset
print(f"Total number of records: {len(eating_behavior_df)}")
print(f"Number of unique patients: {eating_behavior_df['patient_id'].nunique()}")
print(f"Number of unique medical records: {eating_behavior_df['medical_record_id'].nunique()}")

print("\nVerification of no duplicates:")
print(f"Maximum occurrences of any patient_id: {eating_behavior_df['patient_id'].value_counts().max()}")

print("\nNumber of non-null values in key columns:")
print(eating_behavior_df[['patient_id', 'medical_record_id'] + eating_behavior_cols].info())

# Close the connection
conn.close()

# Display the DataFrame
eating_behavior_df

# Optional: Save the DataFrame for later use
eating_behavior_df.to_csv('eating_behavior.csv', index=False)

Total number of records: 1499
Number of unique patients: 1499
Number of unique medical records: 1499

Verification of no duplicates:
Maximum occurrences of any patient_id: 1

Number of non-null values in key columns:
<class 'pandas.core.frame.DataFrame'>
Index: 1499 entries, 384 to 1546
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   patient_id              1499 non-null   object 
 1   medical_record_id       1499 non-null   object 
 2   emotional_eating        1499 non-null   object 
 3   satiety                 1499 non-null   object 
 4   hunger                  1499 non-null   object 
 5   emotional_eating_value  1499 non-null   float64
 6   quantity_control        1499 non-null   float64
 7   impulse_control         1499 non-null   float64
dtypes: float64(3), object(5)
memory usage: 105.4+ KB
None


In [ ]:
eating_behavior_df

Identify patients with eating behavior AND genetics data available - 160 patients

In [8]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('pnk_db2_standard_clean.sqlite')

# Define the columns we want to check for non-NA values
eating_behavior_cols = [
    'emotional_eating',
    'satiety',
    'hunger',
    'emotional_eating_value',
    'quantity_control',
    'impulse_control'
]

# Create the complex SQL query
query = """
WITH 
-- First, get patients with complete eating behavior data
eating_behavior_records AS (
    SELECT medical_record_id
    FROM medical_records_standard_clean
    WHERE emotional_eating IS NOT NULL
    AND satiety IS NOT NULL
    AND hunger IS NOT NULL
    AND emotional_eating_value IS NOT NULL
    AND quantity_control IS NOT NULL
    AND impulse_control IS NOT NULL
),
-- Then, get patients with exactly one medical record
single_record_patients AS (
    SELECT patient_id, medical_record_id
    FROM prescriptions_standard_clean
    GROUP BY patient_id
    HAVING COUNT(DISTINCT medical_record_id) = 1
),
-- Get patients with genomics data
genomics_patients AS (
    SELECT patient_id, genomics_sample_id
    FROM patients_standard_clean
    WHERE genomics_available = 'Sí'
)
-- Combine all conditions
SELECT DISTINCT
    s.medical_record_id,
    s.patient_id,
    g.genomics_sample_id,
    m.*
FROM single_record_patients s
JOIN eating_behavior_records e ON s.medical_record_id = e.medical_record_id
JOIN genomics_patients g ON s.patient_id = g.patient_id
JOIN medical_records_standard_clean m ON s.medical_record_id = m.medical_record_id
"""

# Load the data into a pandas DataFrame
eating_behavior_with_genetics_df = pd.read_sql_query(query, conn)

# Print summary statistics
print("Summary of the final dataset:")
print(f"Total number of records: {len(eating_behavior_with_genetics_df)}")
print(f"Number of unique patients: {eating_behavior_with_genetics_df['patient_id'].nunique()}")
print(f"Number of unique medical records: {eating_behavior_with_genetics_df['medical_record_id'].nunique()}")
print(f"Number of unique genomics samples: {eating_behavior_with_genetics_df['genomics_sample_id'].nunique()}")

# Verify data completeness
print("\nVerifying data completeness:")
print(eating_behavior_with_genetics_df[['patient_id', 'medical_record_id', 'genomics_sample_id'] + eating_behavior_cols].info())

# Close the connection
conn.close()

# Display the first few rows of the DataFrame
eating_behavior_with_genetics_df

# Optional: Save the DataFrame
eating_behavior_with_genetics_df.to_csv('eating_behavior_genetics.csv', index=False)

Summary of the final dataset:
Total number of records: 160
Number of unique patients: 160
Number of unique medical records: medical_record_id    160
medical_record_id    160
dtype: int64
Number of unique genomics samples: 160

Verifying data completeness:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   patient_id              160 non-null    object 
 1   medical_record_id       160 non-null    object 
 2   medical_record_id       160 non-null    object 
 3   genomics_sample_id      160 non-null    object 
 4   emotional_eating        160 non-null    object 
 5   satiety                 160 non-null    object 
 6   hunger                  160 non-null    object 
 7   emotional_eating_value  160 non-null    float64
 8   quantity_control        160 non-null    float64
 9   impulse_control         160 non-null    float64
dty

In [ ]:
eating_behavior_with_genetics_df

Retrieve the weight and body composition measurements of patients with available eating behavior data

In [9]:
import sqlite3
import pandas as pd

# Get list of patient IDs from eating_behavior_df
eating_behavior_patient_ids = eating_behavior_df['patient_id'].tolist()

# Connect to the database
conn = sqlite3.connect('pnk_db2_standard_clean.sqlite')

# First, let's check the table structure
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(measurements_standard_clean)")
columns = cursor.fetchall()
print("Table columns:")
for col in columns:
    print(col[1])  # Print column names

# Create query using the list of patient IDs and correct date column name
query = f"""
SELECT *
FROM measurements_standard_clean
WHERE patient_id IN ('{"','".join(eating_behavior_patient_ids)}')
ORDER BY patient_id, measurement_date
"""

# Load data into DataFrame
eating_behavior_measurements = pd.read_sql_query(query, conn)

# Print summary statistics
print("\nSummary of measurements data:")
print("-" * 50)
print(f"Total number of measurements: {len(eating_behavior_measurements)}")
print(f"Number of unique patients: {eating_behavior_measurements['patient_id'].nunique()}")
print(f"\nColumns in the dataset:")
print(eating_behavior_measurements.info())

# Close connection
conn.close()

# Display the table
eating_behavior_measurements

# Sort the DataFrame by patient_id and measurement_date (if not already sorted)
eating_behavior_measurements = eating_behavior_measurements.sort_values(['patient_id', 'measurement_date'])

# Create an empty row with NaN values and the same columns as the DataFrame
empty_row = pd.DataFrame({col: [None] for col in eating_behavior_measurements.columns})

# Insert an empty row after each group of patient_id
eating_behavior_measurements_with_separator_rows = (
    eating_behavior_measurements.groupby('patient_id', group_keys=True)
    .apply(lambda group: pd.concat([group, empty_row], ignore_index=True))
    .reset_index(drop=True)
)

# Display the modified DataFrame
eating_behavior_measurements_with_separator_rows

Table columns:
patient_id
measurement_date
weight_kg
bmi
bmr_kcal
fat_%
vat_%
muscle_%
water_%

Summary of measurements data:
--------------------------------------------------
Total number of measurements: 25669
Number of unique patients: 1499

Columns in the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25669 entries, 0 to 25668
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   patient_id        25669 non-null  object 
 1   measurement_date  25669 non-null  object 
 2   weight_kg         25669 non-null  float64
 3   bmi               25669 non-null  float64
 4   bmr_kcal          25196 non-null  float64
 5   fat_%             25570 non-null  float64
 6   vat_%             16616 non-null  float64
 7   muscle_%          25669 non-null  float64
 8   water_%           25669 non-null  float64
dtypes: float64(7), object(2)
memory usage: 1.8+ MB
None


C:\Users\Felhasználó\AppData\Local\Temp\ipykernel_22680\1978663348.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  .apply(lambda group: pd.concat([group, empty_row], ignore_index=True))
C:\Users\Felhasználó\AppData\Local\Temp\ipykernel_22680\1978663348.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: pd.concat([group, empty_row], ignore_index=True))


,patient_id,measurement_date,weight_kg,bmi,bmr_kcal,fat_%,vat_%,muscle_%,water_%
0,9896E54CEB969,2022-05-11 11:03:19,96.70,29.85,2021.15,32.37,11.39,34.99,47.63
1,9896E54CEB969,2022-05-12 08:36:04,95.10,29.35,1984.98,32.25,11.10,34.82,47.75
2,9896E54CEB969,2022-05-13 07:57:10,94.40,29.14,1991.37,30.95,10.98,35.88,49.05
3,9896E54CEB969,2022-05-14 07:54:44,93.80,28.95,1964.97,31.63,10.87,35.16,48.37
4,9896E54CEB969,2022-05-15 09:07:36,93.60,28.89,1980.48,30.49,10.84,36.16,49.51
...,...,...,...,...,...,...,...,...,...
27163,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27164,9AA7A14CEB969,2024-02-27 08:49:48,32.07,42.90,12.67,1.00,3.88,1978.22,31.76
27165,9AA7A14CEB969,2024-02-28 08:45:58,31.87,44.35,12.56,1.00,3.94,1989.60,32.97
27166,9AA7A14CEB969,2024-02-29 08:30:53,31.41,45.22,12.30,1.00,3.97,1971.66,33.52


Retrieve the first and last body composition measurement of each patient, and calculate the number of days passed between the two. 

NEEDS REVISION: Currently, this code does not make too much sense, as it does not account for restarters, it does not include the medical record IDs. 

In [10]:
import pandas as pd

# Ensure the measurements are sorted by patient_id and measurement_date.
df = eating_behavior_measurements.sort_values(['patient_id', 'measurement_date'])

# Extract the first and last measurements for each patient.
first_measurements = df.groupby('patient_id', as_index=False).first()
last_measurements = df.groupby('patient_id', as_index=False).last()

# Calculate follow-up length (days) between the first and last measurements.
# For first measurements, followup_length will be 0.
first_measurements['followup_length'] = 0
last_measurements['followup_length'] = (
    pd.to_datetime(last_measurements['measurement_date']) - pd.to_datetime(first_measurements['measurement_date'])
).dt.days

# Combine the first and last measurements into one DataFrame.
eating_behavior_measurements_delta = pd.concat([first_measurements, last_measurements], ignore_index=True)

# Sort again by patient_id and measurement_date to have the first measurement (with 0 days) come before the last.
eating_behavior_measurements_delta = eating_behavior_measurements_delta.sort_values(['patient_id', 'measurement_date'])

# Reorder columns so that 'followup_length' is the third column.
columns = eating_behavior_measurements_delta.columns.tolist()
# Assuming the first two columns are 'patient_id' and 'measurement_date'
new_order = [columns[0], columns[1], 'followup_length'] + [col for col in columns if col not in [columns[0], columns[1], 'followup_length']]
eating_behavior_measurements_delta = eating_behavior_measurements_delta[new_order]

# Display summary
print("\nSummary of first/last measurements data:")
print("-" * 50)
print(f"Total number of rows: {len(eating_behavior_measurements_delta)}")
print(f"Number of unique patients: {eating_behavior_measurements_delta['patient_id'].nunique()}")

# Display first few rows
eating_behavior_measurements_delta


Summary of first/last measurements data:
--------------------------------------------------
Total number of rows: 2998
Number of unique patients: 1499


,patient_id,measurement_date,followup_length,weight_kg,bmi,bmr_kcal,fat_%,vat_%,muscle_%,water_%
0,9896E54CEB969,2022-05-11 11:03:19,0,96.70,29.85,2021.15,32.37,11.39,34.99,47.63
1499,9896E54CEB969,2022-05-21 08:47:06,9,28.40,50.19,10.55,2.00,10.66,1954.39,36.50
1,9896F54CEB969,2023-01-17 06:05:55,0,26.40,55.46,9.41,2.00,4.29,1677.59,39.56
1500,9896F54CEB969,2023-10-12 07:15:08,268,25.05,57.58,8.63,1.00,4.50,1622.11,40.81
2,9897484CEB969,2022-04-24 08:02:47,0,63.90,26.60,1332.45,34.76,5.20,34.77,43.43
...,...,...,...,...,...,...,...,...,...,...
2995,9AA6C74CEB969,2024-02-21 10:53:57,0,26.59,41.69,5.20,1.00,4.05,1471.55,31.69
1497,9AA7064CEB969,2024-02-25 09:07:05,0,28.29,40.10,5.53,1.00,4.20,1351.67,32.01
2996,9AA7064CEB969,2024-02-29 08:38:19,3,27.62,41.27,5.40,1.00,4.29,1330.71,32.75
1498,9AA7A14CEB969,2024-02-27 08:49:48,0,32.07,42.90,12.67,1.00,3.88,1978.22,31.76
